In [1]:
import sys
from scipy.sparse import load_npz, save_npz
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp

import utils
from baselines import (Forge_Adjacency, Forge_Modularity, Forge_SymmetricLaplacian, Forge_Transition,
                       train_and_log_multiple_conf_models)

In [2]:
_A_obs = load_npz('../data/datasets/CORA_ML.npz')

In [3]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()
val_edges = (val_ones, val_zeros)

## Forge Framework

In [4]:
fa = Forge_Adjacency(A=train_graph, rank=1600)
graphs_fa = fa(sample_size=5,
               logdir='../logs/CORA-ML/baseline_FA',
               val_edges=val_edges)

In [5]:
ft = Forge_Transition(A=train_graph, rank=1600)
graphs_ft = ft(sample_size=5,
               logdir='../logs/CORA-ML/baseline_FT',
               val_edges=val_edges)

In [6]:
fm = Forge_Modularity(A=train_graph, rank=1600)
graphs_fm = fm(sample_size=5,
               logdir='../logs/CORA-ML/baseline_FM',
               val_edges=val_edges)

In [8]:
fs = Forge_SymmetricLaplacian(A=train_graph, rank=2500)
graphs_fs = fs(sample_size=5,
               logdir='../logs/CORA-ML/baseline_FS',
               val_edges=val_edges)

## Configuration Model

In [9]:
train_and_log_multiple_conf_models(logdir='../logs/CORA-ML/baseline_conf/',
                                   A=train_graph,
                                   EOs=[0.3, 0.5, 0.7],
                                   experiments_per_EO=20)